# Simple application of Arcs to a toy example

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

from dynamicComputation import *
from scoring import *
from syntheticData import *
from lengthPriors import *

Let's get some default priors for the arcs.

In [ ]:
from defaultPriors import arcPriorTempo as arcPrior

In [ ]:
%%script false --no-raise-error
# Old way of having a length prior

pmf = sp.stats.binom.pmf(list(range(31)),30,.5)
pmf[0] = 0
pmf[1] = 0
pmf = [float(m)/sum(pmf) for m in pmf]

cmf = np.cumsum(pmf)

def priorEvalCond(N,a,b):
    pmf = sp.stats.binom.pmf(list(range(31)),30,.5)
    pmf[0] = 0
    pmf[1] = 0
    pmf = [float(m)/sum(pmf) for m in pmf]
    cmf = np.cumsum(pmf)

    maxLength = 30
    cum = 1 if N-a > maxLength or cmf[N-a]==0 else cmf[N-a]
    prob = 1 if b-a > maxLength else pmf[b-a]
    return prob/cum

# Assume 10% probability of arc ending at any point
def priorEval(N,a,b):
    return .1*.1*pow(1-.1,b-a-1)


lengthPrior = {
    # Free prior on the length of arcs
    # needs to be scaled to the range of possible values
    'max': 30,  # Any longer will have 0 likelihood -- useful for bounding complexity
    'cmf':cmf,
    'pmf':pmf,
    # 'cmf': [0, 0, .05, .20, .35, .65, .80, .95, 1],  # Cumulative mass function
    # 'pmf': [0, 0, .05, .15, .15, .3, .15, .15, .05]  # Probability mass function
    'eval': priorEval,
    'evalCond': priorEvalCond 
}


In [ ]:
#Generate some segments
rng = np.random.default_rng()
testSegment = [math.ceil(x) for x in rng.normal(15,5,size=10)]
print(testSegment)

In [ ]:
#Generate the actual data
(truth, sampleData) = genData(testSegment, arcPrior)
print(len(sampleData),sampleData)

In [ ]:
#Bind a length prior to our data
lengthPrior = NormalLengthPrior(mean=15,stddev=5,x=list(range(len(sampleData))),maxLength=30)

In [ ]:
#Compute posterior marginals
posteriorMarginals = runAlphaBeta(sampleData,arcPrior,lengthPrior)

In [ ]:
#Display the results
fig, ax1 = plt.subplots()
ax1.plot(sampleData)
ax1.plot(truth)
#print(posteriorMarginals)

#Using the guess
ax2 = ax1.twinx()

ax2.plot(posteriorMarginals, 'r')

plt.show()

In [ ]:

guesses, convol = marginal2guess(posteriorMarginals[1:],3,.5)
ax2.plot(convol, 'g')
ax2.vlines(guesses,ymin=0,ymax=1)
plt.show()